In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.apps.borg.hive import VaspToComputedEntryDrone
from pymatgen.apps.borg.queen import BorgQueen
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter

# Assimilate VASP calculations into ComputedEntry object. Let's assume that
# the calculations are for a series of new LixFeyOz phases that we want to
# know the phase stability.

In [ ]:
drone = VaspToComputedEntryDrone()
queen = BorgQueen(drone, rootpath=".")
entries = queen.get_data()

In [ ]:
# Obtain all existing Li-Fe-O phases using the Materials Project REST API
with MPRester("P2tLmbTfRg9uMJDqzXV") as m:
    mp_entries = m.get_entries_in_chemsys(["Ni","Fe"])


In [ ]:
entry = mp_entries[20]
entry

In [ ]:
type(entry.entry_id)

In [ ]:
from pymatgen import MPRester
from pymatgen.analysis.diffraction.xrd import XRDCalculator

mpr = MPRester("P2tLmbTfRg9uMJDqzXV")

structure = mpr.get_structure_by_material_id("mp-2213")

# Defaults to Cu K_alpha radiation, no symmetry refinement, no Debye-Waller factors
xrd_calculator = XRDCalculator()

# Defaults to return scaled (peak value 100) intensities with two_theta_range (0, 90)
pattern = xrd_calculator.get_pattern(structure, two_theta_range=(0, 180))

In [ ]:
import matplotlib.pyplot as plt

plt.bar(pattern.x, pattern.y)

In [ ]:
import requests
material_id = "mp-2213"
rad_source = "Cu"

rv = requests.get(
    'https://materialsproject.org/materials/{material_id}/xrd?symbol={rad_source}'.format(
        material_id=material_id,
        rad_source=rad_source,
    ))
try:
    data = rv.json()
except:
    print("Error. Perhaps unknown material id or radiation source.")

In [ ]:
import pandas as pd

df = pd.DataFrame(data["pattern"], columns = data["meta"])
plt.bar(df.two_theta, df.amplitude)

In [ ]:
from apimanage import somesomesome
hehe = somesomesome("P2tLmbTfRg9uMJDqzXV")
hehe.get_entries_by_composition(["Ni", "Fe"])
dic = hehe.xrd_dictionary

In [ ]:
import numpy as np
import plotly.graph_objects as go
def show_XRD_graph(dictionary):
    if dictionary != {}:
        fig = go.Figure()
        mp_id_list = list(dictionary.keys())
        for id in mp_id_list:
            dff = dictionary[id]
            x = dff["two_theta"].to_numpy()
            y = dff["amplitude"].to_numpy()
            fig.add_trace(go.Bar(visible= False, x=x, y=y, width = 2,opacity=0.8,
                                 marker=dict(color='red',
                                             line=dict(width=0))))
        fig.data[0].visible = True
        steps = []
        for i in range(len(fig.data)):
            step = dict(
                method="update",
                args=[{"visible": [False] * len(fig.data)}],  # layout attribute
                label=mp_id_list[i]
            )
            step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
            steps.append(step)
        sliders = [dict(
            active=0,
            steps=steps
        )]
        fig.update_yaxes(scaleanchor="x", scaleratio=1)
        fig.update_layout(
            xaxis_title="X",
            yaxis_title="Y",
            template='plotly_white',
            sliders=sliders,
            xaxis_range=[0,180]
        )
        fig.show()

In [ ]:
show_XRD_graph(dic)

In [ ]:
import plotly
def fig_to_html(fig):
    config = config={'doubleClick': 'reset', 'displaylogo': False, 'scrollZoom': True}
    html = '<html><head><meta charset="utf-8"/><script src="https://cdn.plot.ly/plotly-latest.min.js"></script></head><body>'
    html += plotly.offline.plot(fig, output_type='div', include_plotlyjs='cdn', config = config)
    html += '</body></html>'
    return html

In [ ]:
import plotly.express as px

sss = dic['mp-1245108']
x = sss["two_theta"].to_numpy()
figure = px.scatter(sss, x="two_theta", y = "amplitude")
figure.update_traces(marker_color='red',  opacity=0.6)
figure.show()